In [1055]:
import pandas as pd
import numpy as np

In [1056]:
player_stats = pd.read_csv("merged_df.csv").drop('Unnamed: 0', axis='columns')

In [1057]:
play_data = pd.read_csv("./data2023/plays.csv")

In [1058]:
player_stats.head()

,Acceleration,Age,Agility,Archetype,Awareness,BC Vision,Block Shedding,Break Sack,Break Tackle,Carrying,...,Zone Coverage,birthDate,collegeName,displayName,height,nflId,officialPosition,updatedName,updatedName2,weight
0,66,44,70,QB_FieldGeneral,99,51,33,61,31,69,...,19,1977-08-03,Michigan,Tom Brady,6-4,25511,QB,Tom Brady,Tom Brady,225
1,77,39,67,QB_FieldGeneral,85,68,45,83,64,60,...,13,1982-03-02,"Miami, O.",Ben Roethlisberger,6-5,28963,QB,Ben Roethlisberger,Ben Roethlisberger,240
2,84,37,81,QB_Improviser,96,78,19,86,65,63,...,19,1983-12-02,California,Aaron Rodgers,6-2,29851,QB,Aaron Rodgers,Aaron Rodgers,225
3,79,38,71,QB_FieldGeneral,82,73,12,82,66,63,...,12,1982-11-24,Harvard,Ryan Fitzpatrick,6-2,30078,QB,Ryan Fitzpatrick,Ryan Fitzpatrick,228
4,80,37,72,TE_Blocking,76,75,45,11,54,78,...,13,1984-05-19,UCLA,Marcedes Lewis,6-6,30842,TE,Marcedes Lewis,Marcedes Lewis,267


In [1059]:
player_stats.Team.unique()

array(['Buccaneers', 'Steelers', 'Packers', 'Football Team', 'Rams',
       'Falcons', 'Seahawks', 'Ravens', 'Saints', '49ers', 'Chargers',
       'Dolphins', 'Eagles', 'Broncos', 'Patriots', 'Bills', 'Titans',
       'Texans', 'Bears', 'Cowboys', 'Cardinals', 'Vikings', 'Giants',
       'Browns', 'Lions', 'Bengals', 'Raiders', 'Colts', 'Jaguars',
       'Panthers', 'Chiefs', 'Jets'], dtype=object)

In [1060]:
# Get name to match play_data format
player_stats["abrev_name"] = player_stats.displayName.apply(lambda x: str(x)[0]+ "."+"".join(str(x).split(" ")[1:]))
player_stats.abrev_name = player_stats.abrev_name.apply(lambda x: str(x).lower())

In [1061]:
player_stats.abrev_name

0                t.brady
1       b.roethlisberger
2              a.rodgers
3          r.fitzpatrick
4                m.lewis
              ...       
1409         j.jefferson
1410             d.milne
1411            g.stuard
1412            j.jacobs
1413            j.heflin
Name: abrev_name, Length: 1414, dtype: object

In [1062]:
team_mapping = {"PHI": "Eagles", "BUF": "Bills", "ATL": "Falcons", "PIT": "Steelers", "CLE": "Browns", "CIN": "Bengals", "IND": "Colts", "TEN": "Titans", "MIA": "Dolphins", "BAL": "Ravens", "NE": "Patriots", "HOU": "Texans", "JAX": "Jaguars", "NYG": "Giants", "NO": "Saints", "TB": "Buccaneers", "WAS": "Commanders", "ARI": "Cardinals", "CAR": "Panthers", "DAL": "Cowboys", "GB": "Packers", "CHI": "Bears", "NYJ": "Jets", "DET": "Lions", "OAK": "Raiders", "LA": "Rams", "MIN": "Vikings", "LAC": "Chargers", "KC": "Chiefs", "SF": "49ers", "DEN": "Broncos", "SEA": "Seahawks", "LV": "Raiders"}

In [1063]:
play_data['Team'] = play_data.possessionTeam.apply(lambda x: team_mapping[str(x)])

In [1064]:
play_data.possessionTeam.unique()

array(['TB', 'DAL', 'ATL', 'PHI', 'BUF', 'PIT', 'NYJ', 'CAR', 'MIN',
       'CIN', 'DET', 'SF', 'HOU', 'JAX', 'IND', 'SEA', 'TEN', 'ARI',
       'LAC', 'WAS', 'CLE', 'KC', 'NE', 'MIA', 'NO', 'GB', 'NYG', 'DEN',
       'CHI', 'LA', 'LV', 'BAL'], dtype=object)

In [1065]:
# Remove time from play description
playDescText = play_data.playDescription.apply(lambda x: " ".join(str(x).split()[1:]))
# Remove formation from play description
playDescText = playDescText.apply(lambda x: "".join(x.split(")")[1:]) if str(x).startswith("(") else x)
playDescText

0         T.Brady pass incomplete deep right to C.Godwin.
1        D.Prescott pass deep left to A.Cooper pushed ...
2        D.Prescott pass short middle to D.Schultz to ...
3         D.Prescott pass incomplete deep left to C.Lamb.
4        D.Prescott pass incomplete short left to C.La...
                              ...                        
8552     P.Mahomes sacked at NYG 16 for -8 yards (K.Cr...
8553     D.Jones pass short right to E.Engram pushed o...
8554      D.Jones sacked at NYG 20 for -8 yards (C.Jones.
8555     D.Jones pass incomplete short right to E.Engram.
8556     D.Jones sacked at NYG 14 for -6 yards (F.Clar...
Name: playDescription, Length: 8557, dtype: object

In [1066]:
# People who received the ball
receivers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("to")+1] if ("pass" in str(x).lower().split(" ") and "to" in str(x).lower().split(" ")) & ("intercept" not in str(x).lower()) else np.nan)
# Remove full stop from some names at end
receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."] = receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."].apply(lambda x: "".join(str(x)[:-1]))
# 2 values have NO # instead of names, so we can remove these
receivers.loc[receivers=="no"] = np.nan

In [1067]:
play_data['abrev_name'] = receivers # column representing who received
receiver_stats = play_data.merge(player_stats, how='left', left_on=['Team','abrev_name'], right_on=['Team','abrev_name'])
receiver_stats = receiver_stats.rename(columns={'abrev_name': "receiver"})

In [1068]:
receiver_stats.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,Zone Coverage,birthDate,collegeName,displayName,height,nflId,officialPosition,updatedName,updatedName2,weight
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,25.0,1996-02-27,Penn State,Chris Godwin,6-1,44896.0,WR,Chris Godwin,Chris Godwin,209.0
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,25.0,1994-06-17,Alabama,Amari Cooper,6-1,42347.0,WR,Amari Cooper,Amari Cooper,211.0
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,15.0,1996-07-11,Stanford,Dalton Schultz,6-5,46206.0,TE,Dalton Schultz,Dalton Schultz,242.0
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,25.0,1999-04-08,Oklahoma,CeeDee Lamb,6-2,52425.0,WR,CeeDee Lamb,CeeDee Lamb,189.0
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,25.0,1999-04-08,Oklahoma,CeeDee Lamb,6-2,52425.0,WR,CeeDee Lamb,CeeDee Lamb,189.0


In [1069]:
#Drop non-Madden players and anomalies
receiver_stats.dropna(axis='rows', subset='nflId', inplace=True)
receiver_stats.reset_index(drop=True, inplace=True)

In [1070]:
len(receiver_stats['nflId'].unique())

336

In [1071]:
feature_columns = ['Age', 'Height', 'Weight', 'Overall', 'Speed',
 'Acceleration', 'Agility', 'Change of Dir', 'Strength', 'Jumping',
 'Awareness', 'Carrying', 'Break Tackle', 'Juke Move', 'Spin Move',
 'Trucking', 'Stiff Arm', 'BC Vision', 'Catching', 'Catch In Traffic',
 'Spec Catch', 'Release', 'Short RR', 'Medium RR', 'Deep RR',
 'Throw Power', 'Throw Acc Short', 'Throw Acc Mid', 'Throw Acc Deep',
 'Throw Under Pressure', 'Throw On The Run', 'Play Action', 'Break Sack',
 'Run Block', 'Run Block Power', 'Run Block Finesse', 'Pass Block',
 'Pass Block Power', 'Pass Block Finesse', 'Impact Blocking',
 'Lead Blocking', 'Tackle', 'Hit Power', 'Pursuit', 'Man Coverage',
 'Zone Coverage', 'Press', 'Play Recognition', 'Power Moves',
 'Finesse Moves', 'Block Shedding', 'Kick Power', 'Kick Accuracy',
 'Kick Return', 'Stamina', 'Injury', 'Toughness', 'Years Pro']

In [1072]:
total_yards_by_stats = pd.DataFrame({'nflId': receiver_stats['nflId'].astype('int32').unique()})

In [1073]:
total_yards_by_stats['total_yards'] = total_yards_by_stats['nflId'].map(lambda x: receiver_stats[receiver_stats['nflId'] == x]['prePenaltyPlayResult'].sum())

In [1074]:
total_yards_by_stats = total_yards_by_stats.merge(player_stats[feature_columns + ['nflId']], how='inner', on='nflId')
total_yards_by_stats.head()

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
0,44896,535,25,73,205,87,91,92,91,89,...,10,10,28,25,21,48,88,96,90,4
1,42347,460,27,73,210,92,91,89,95,94,...,10,10,35,25,21,77,94,83,93,6
2,46206,332,25,77,244,74,81,87,81,71,...,10,10,31,16,14,38,79,91,86,3
3,52425,524,22,74,189,81,90,90,90,89,...,10,10,30,48,33,80,90,93,91,1
4,41233,538,28,77,231,91,90,89,84,83,...,10,10,35,25,24,55,97,91,94,7


In [1075]:
total_yards_by_stats.describe()

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,336.00000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,46523.086310,145.324405,25.729167,73.086310,217.517857,74.851190,87.315476,89.223214,85.306548,81.485119,...,14.895833,15.500000,33.854167,21.074405,18.12500,56.202381,84.782738,87.616071,83.241071,3.559524
std,4763.769627,157.109522,2.946465,2.991778,25.872356,8.540083,5.228098,3.125370,5.609008,7.426038,...,10.329115,10.639549,9.980556,7.825168,7.05855,25.196555,4.711098,3.436187,6.001361,2.763736
min,30842.000000,-1.000000,20.000000,65.000000,170.000000,61.000000,62.000000,78.000000,66.000000,54.000000,...,5.000000,5.000000,10.000000,11.000000,10.00000,6.000000,75.000000,75.000000,63.000000,0.000000
25%,43389.500000,30.000000,23.000000,71.000000,199.750000,68.000000,85.000000,87.750000,82.000000,77.000000,...,10.000000,10.000000,26.000000,17.750000,14.00000,35.000000,81.750000,86.000000,79.000000,1.000000
50%,46114.500000,95.500000,25.000000,73.000000,212.500000,74.000000,88.000000,90.000000,85.000000,82.500000,...,10.000000,10.000000,32.000000,20.000000,16.00000,65.000000,84.000000,88.000000,84.000000,3.000000
75%,52429.250000,195.750000,28.000000,76.000000,238.000000,80.000000,91.000000,91.000000,89.000000,87.000000,...,16.250000,17.000000,40.000000,22.000000,20.00000,78.000000,87.000000,90.000000,87.000000,5.000000
max,53686.000000,793.000000,37.000000,80.000000,320.000000,99.000000,99.000000,99.000000,98.000000,99.000000,...,74.000000,66.000000,72.000000,75.000000,62.00000,98.000000,98.000000,97.000000,98.000000,15.000000


In [1076]:
X = total_yards_by_stats[feature_columns]
X.shape

(336, 58)

In [1077]:
Y = total_yards_by_stats['total_yards']
Y.shape

(336,)

In [1078]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=0)

In [1079]:
select_k_best=False

In [1080]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

if select_k_best:
    k_best = SelectKBest(mutual_info_regression, k=20).fit(X_train, Y_train)
    X_train = k_best.transform(X_train)
    X_test = k_best.transform(X_test)
    print(k_best.get_feature_names_out())

In [1081]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, Y_train)
reg.score(X_test, Y_test)

array([ 4.09162667e+00,  7.41909581e-01,  1.67549333e+00,  6.55823152e+00,
        5.67943573e-01,  3.26101448e+00, -6.81961682e-01,  2.13279391e+00,
       -2.65873572e+00, -3.99984430e-01,  1.66628458e+00, -4.91480924e+00,
       -3.19880890e+00, -4.18814847e+00,  3.73786120e+00,  1.78145084e+00,
       -1.53244554e-01,  3.07903909e+00,  1.26173977e+00,  4.12307680e-01,
       -3.62498961e+00, -5.10093255e-01,  4.41998256e+00,  2.07028111e+00,
        1.35294840e+00,  1.65604520e+00, -7.46686664e-01,  5.38555952e+00,
       -3.26674822e+00, -5.48633998e+00, -1.88951728e+00,  1.18279515e-01,
        3.82054265e+00,  2.22432162e+00,  1.18586777e-02,  3.90023862e-01,
        1.34140839e+00, -2.74518896e+00,  1.08058901e+00,  4.89936705e-02,
       -2.18433901e+00,  1.59088951e+00,  1.21751418e+00, -2.03724628e+00,
        6.78879603e-01,  2.11353644e+00, -8.43771987e-01, -8.12633197e-01,
       -1.64031341e+00, -9.65149273e-01,  3.90846909e-01, -1.72473191e+00,
        3.67672701e+00,  

In [1082]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,152.306762
std,124.479756
min,-39.166241
25%,69.789803
50%,142.984015
75%,219.452580
max,521.766511


In [1083]:
from sklearn import svm
reg = svm.SVR().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.0869098369029111

In [1084]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,92.987533
std,3.425767
min,86.100971
25%,90.418853
50%,92.795054
75%,95.146309
max,101.334767


In [1085]:
from sklearn import tree
reg = tree.DecisionTreeRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.051905273900092475

In [1086]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,142.264706
std,148.851586
min,-1.000000
25%,33.500000
50%,83.500000
75%,205.500000
max,653.000000


In [1087]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.3444969969210221

In [1088]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,149.093088
std,113.537923
min,32.050000
25%,75.547500
50%,99.420000
75%,184.447500
max,517.680000


In [1089]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(learning_rate_init = 0.01, learning_rate='adaptive').fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.34904105439932154

In [1090]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,182.669073
std,124.536319
min,26.748935
25%,102.690883
50%,142.097553
75%,224.216571
max,617.434303
